**This is an example from Deep Learning With Python book.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

## Loading the dataset

In [2]:
from keras.datasets import boston_housing

(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


57344/57026 [==============================] - 0s 0us/step


In [3]:
# take a look at the data

print(f'Training data : {train_data.shape}')
print(f'Test data : {test_data.shape}')
print(f'Training sample : {train_data[0]}')
print(f'Training target sample : {train_targets[0]}')

Training data : (404, 13)
Test data : (102, 13)
Training sample : [  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]
Training target sample : 15.2


## Preparing the data

We are going to do a feature normalization . Feature normalizaion is when you subtract the mean of the feature from each feature and divide each result by the standard deviation.


In [4]:
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

# Note that the quantities used for normalizing the test data are computed using the
# training data. You should never use in your workflow any quantity computed on the
# test data, even for something as simple as data normalization.

## Building the network

In [5]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))

    model.compile(optimizer='rmsprop',
              loss='mse',
              metrics=['mae'])
    return model

## K-fold validation

In [6]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print(f'Processing fold # {i}')
    val_data = train_data[i * num_val_samples: (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples: (i+1) * num_val_samples]
    
    partial_train_data = np.concatenate(
                            [train_data[:i * num_val_samples],
                            train_data[(i+1) * num_val_samples:]],
                            axis=0)
    partial_train_targets = np.concatenate(
                            [train_targets[:i * num_val_samples],
                            train_targets[(i+1)*num_val_samples:]],
                            axis=0)
    model = build_model()
    model.fit(partial_train_data,
              partial_train_targets,
              epochs=num_epochs,
              batch_size=1,
              verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

Processing fold # 0
Processing fold # 1
Processing fold # 2
Processing fold # 3


In [7]:
print(f'all_scores : {all_scores}')
print(f'mean all scores : {np.mean(all_scores)}')

all_scores : [2.1299970468672194, 2.2819440813347844, 2.9202968838191268, 2.3692543305973017]
mean all scores : 2.4253730856546083


In [8]:
model = build_model()
model.fit(train_data, train_targets, epochs= 80, batch_size= 16, verbose= 1)
test_mse_score, test_mae_score = model.evaluate(test_data, test_targets)

Epoch 1/80
404/404 [==============================] - 0s 644us/step - loss: 469.1542 - mean_absolute_error: 19.7320
Epoch 2/80
404/404 [==============================] - 0s 72us/step - loss: 277.9006 - mean_absolute_error: 14.5058
Epoch 3/80
404/404 [==============================] - 0s 62us/step - loss: 125.1571 - mean_absolute_error: 8.8875
Epoch 4/80
404/404 [==============================] - 0s 66us/step - loss: 62.0649 - mean_absolute_error: 6.0280
Epoch 5/80
404/404 [==============================] - 0s 65us/step - loss: 37.5030 - mean_absolute_error: 4.6045
Epoch 6/80
404/404 [==============================] - 0s 65us/step - loss: 26.3543 - mean_absolute_error: 3.7328
Epoch 7/80
404/404 [==============================] - 0s 66us/step - loss: 20.7812 - mean_absolute_error: 3.1800
Epoch 8/80
404/404 [==============================] - 0s 67us/step - loss: 18.1548 - mean_absolute_error: 2.9676
Epoch 9/80
404/404 [==============================] - 0s 64us/step - loss: 16.7979 - mean_

In [9]:
test_mae_score

3.019854003307866